In [13]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 5.8MB/s 
     |████████████████████████████████| 7.4MB 36.7MB/s 
     |████████████████████████████████| 2.9MB 45.5MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 317kB 39.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.8MB 41.1MB/s 
     |████████████████████████████████| 1.4MB 39.7MB/s 
     |████████████████████████████████| 163kB 29.6MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 4.5MB 41.4MB/s 
     |████████████████████████████████| 112kB 43.9MB/s 
     |████████████████████████████████| 133kB 39.9MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 890kB 37.8MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 122kB 44.6MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
data = pd.read_csv("ner_dataset.csv",encoding="latin1" )

In [2]:
data =data.fillna(method ="ffill")

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [5]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [6]:
data["labels"] = data["labels"].str.upper()

In [7]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [9]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [10]:
from simpletransformers.ner import NERModel,NERArgs

In [11]:
label = data["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [12]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [13]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [14]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1499, 0.1938556114670195)

In [15]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [16]:
result

{'eval_loss': 0.17011943687270203,
 'f1_score': 0.794500409295177,
 'precision': 0.8264919537033819,
 'recall': 0.7648932063818837}

In [34]:
prediction, model_output = model.predict(["NickBaguley"])

In [35]:
prediction

[[{'NickBaguley': 'B-PER'}]]

In [36]:
import joblib

In [37]:
filename = 'Bert_NER_Model.sav'
joblib.dump(model, filename)

['Bert_NER_Model.sav']